#### Objective of this kernel:
* to further explore the data
* To clean the out put the Dataset to be fed into the various machine algorithms

In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the "../output/" directory.
import os
for dirname, _, filenames in os.walk('./output/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

./output/linearmodel.pickle
./output/processed_data.csv
./output/cleaned_data.csv
./output/.ipynb_checkpoints/cleaned_data-checkpoint.csv


## Reading the data set

In [6]:
df = pd.read_csv('./output/cleaned_data.csv')
df

,Age,Gender,wassce grade,level,current CGPA,access to a laptop or internet,study group,time spent on independent studies
0,23 - 27 years,Male,16 - 20,L 400,2.9 - 2.5,Yes,Yes,More than 2 hours
1,23 - 27 years,Male,6 - 9,L 400,1.9 - 1.5,Yes,Yes,More than 2 hours
2,23 - 27 years,Male,10 - 15,L 400,3.5 - 3.0,Yes,Yes,Less than 2 hours
3,23 - 27 years,Female,10 - 15,L 400,2.9 - 2.5,Yes,Yes,More than 2 hours
4,23 - 27 years,Female,10 - 15,L 400,2.9 - 2.5,Yes,Yes,More than 2 hours
...,...,...,...,...,...,...,...,...
233,18 - 22 years,Male,6 - 9,L 400,4.0 - 3.6,Yes,Yes,More than 2 hours
234,18 - 22 years,Male,6 - 9,L 300,Below 1.5,Maybe,No,More than 2 hours
235,More than 32 years,Female,Above 30,L 300,Below 1.5,No,Yes,Less than 2 hours
236,18 - 22 years,Female,16 - 20,L 300,3.5 - 3.0,Yes,Yes,More than 2 hours


## we are going to transform all our categorical data to proper format to be fed to our machine learning algorithms
* We preprocess our data with the LabelEncoder from sklearn

#### Preprocessing our independent variable x

In [7]:
y= df[["current CGPA"]].values
x =df[["Age","Gender","wassce grade","level","access to a laptop or internet","study group","time spent on independent studies"]].values

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder_x = LabelEncoder()
x[:,0] =label_encoder_x.fit_transform(x[:,0])
x[:,1] =label_encoder_x.fit_transform(x[:,1])
x[:,2] =label_encoder_x.fit_transform(x[:,2])
x[:,3] =label_encoder_x.fit_transform(x[:,3])
x[:,4] =label_encoder_x.fit_transform(x[:,4])
x[:,5] =label_encoder_x.fit_transform(x[:,5])
x[:,6] =label_encoder_x.fit_transform(x[:,6])

#### Preocessing our dependent variable y

In [9]:
y[:,0] =label_encoder_x.fit_transform(y[:,0])
# y

# Lets start implementing our machine learning algorithms

In [10]:
from sklearn.model_selection import train_test_split
import pickle

## Impleamenting the linear regression algorithm
* we train our data 100 iterations 
* we pick the best result

In [12]:
# import the linear regression module from sklearn
best = 0
for _ in range(100):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4)
    from sklearn import linear_model
    linear =linear_model.LinearRegression()
    linear.fit(x_train,y_train)
    linear_score = linear.score(x_test,y_test)
    if linear_score > best:
        best = linear_score
        with open("./output/linearmodel.pickle","wb") as f:
            pickle.dump(linear,f)

#### saving linear regression model 

In [13]:
pickle_in = open("./output/linearmodel.pickle","rb")
linear = pickle.load(pickle_in)

In [17]:
# outputting more info about our linear regression model
print('coeffitient:\n',linear.coef_)
print('Intercept:\n',linear.intercept_)
# variance score: 1 means perfect prediction
print('Variance score: {}'.format(linear.score(x_test, y_test)))

coeffitient:
 [[-0.01234105 -0.08503763  0.16844458 -0.00753245  0.05004587  0.31243237
   0.10100223]]
Intercept:
 [1.91858335]
Variance score: 0.14841180931672093
